Evaluate the Gaussian Process with only location input.

In [1]:
import torch
import gpytorch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
from tqdm import tqdm

from utils import average_hour

# Load Data

In [4]:
data_dir = "../InterpolationBaseline/data/Oct0123_Jan3024/"
data_files = [file for file in os.listdir(data_dir) if file.endswith(".csv")]
data = []
for i, file in enumerate(data_files):
    df = pd.read_csv(data_dir + file)
    df = average_hour(df)

    # remove sensors with missing data
    if len(df) < 2928:
        print("File{}: {} contains missing hours".format(i, file))
        continue

    # remove sensors with outliers
    if df["pm25"].max() > 500:
        print("File{}: {} contains outliers".format(i, file))
        continue

    data.append(df.loc[:, ["pm25", "longitude", "latitude"]])

data = np.array(data).transpose(1, 0, 2)
X = data[:, :, 1:]
Y = data[:, :, 0]
print(X.shape, Y.shape)

File1: 6kzhfU9xTKCUVJMz492l2g.csv contains outliers
File8: OfAvTbS1SiOjQo4WKSAP9g.csv contains missing hours
File17: wSo2iRgjT36eWC4a2joWZg.csv contains outliers
File19: jDYxIP2JQL2br5aTIAR7JQ.csv contains outliers
File21: 4XEJFVFOS761cvyEjOYf0g.csv contains outliers
File22: 3BAGEmnnQ2K4zF49Dkkoxg.csv contains missing hours
File26: 6nBLCf6WT06TOuUExPkBtA.csv contains missing hours
File36: R2ebpAblQHylOjteA-2hlQ.csv contains missing hours
File43: JQ1px-xqQx-xKh3Oa5h9nA.csv contains missing hours
File50: kyRUtBOTTaK7V_-dxOJTwg.csv contains outliers
(2928, 41, 2) (2928, 41)


# Construct GP Model

In [5]:
class GlobalKernel(gpytorch.kernels.Kernel):
    def __init__(self, matern_ard=None, **kwargs):
        super(GlobalKernel, self).__init__(**kwargs)
        if matern_ard is not None:
            self.maternkernel = gpytorch.kernels.MaternKernel(nu=0.5, ard_num_dims=matern_ard)
        else:
            self.maternkernel = gpytorch.kernels.MaternKernel(nu=0.5)
        
        self.scalekernel = gpytorch.kernels.ScaleKernel(self.maternkernel)

    def forward(self, x1, x2, **params):
        return self.scalekernel(x1, x2, **params)
    
class AirGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, matern_ard=None):
        super(AirGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = GlobalKernel(matern_ard=matern_ard)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# LOOCV

In [9]:
n_steps, n_stations, n_features = X.shape
print(n_steps, n_stations, n_features)

2928 41 2


In [12]:
Y_true_all = []
Y_pred_all = []
for t in tqdm(np.arange(n_steps)):
    try:
        np.random.seed(42)

        # leave one out split
        i = np.random.randint(n_stations)
        X_train = torch.from_numpy(np.concatenate([X[t, :i], X[t, i+1:]], axis=0)).float()
        Y_train = torch.from_numpy(np.concatenate([Y[t, :i], Y[t, i+1:]], axis=0)).float()
        X_test = torch.from_numpy(X[t, i:i+1]).float()
        Y_test = torch.from_numpy(Y[t, i:i+1]).float()

        # prepare training
        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        model = AirGP(X_train, Y_train, likelihood, matern_ard=n_features)
        training_iter = 10000
        model.train()
        likelihood.train()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
        
        # training
        for iter in range(training_iter):
            model.train()
            likelihood.train()
            optimizer.zero_grad()
            output = model(X_train)
            loss = -mll(output, Y_train)
            loss.backward()
            optimizer.step()

        # evaluation
        model.eval()
        likelihood.eval()
        Y_pred = model(X_test)
        Y_pred = Y_pred.mean.detach()
        Y_true_all.append(Y_test)
        Y_pred_all.append(Y_pred)
    except Exception as e:
        print("Failed at step {}".format(t))
        continue

Y_true_all = torch.cat(Y_true_all, dim=0).numpy()
Y_pred_all = torch.cat(Y_pred_all, dim=0).numpy()

  0%|          | 5/2928 [00:39<5:40:18,  6.99s/it]

Failed at step 4


  3%|▎         | 93/2928 [14:20<7:00:03,  8.89s/it]

Failed at step 92


  4%|▍         | 123/2928 [18:48<5:19:37,  6.84s/it]

...

 94%|█████████▍| 2745/2928 [7:17:02<18:27,  6.05s/it]

Failed at step 2744


 96%|█████████▌| 2816/2928 [7:27:43<13:30,  7.24s/it]

Failed at step 2815


100%|██████████| 2928/2928 [7:44:46<00:00,  9.52s/it]


In [13]:
rmse = np.sqrt(np.mean((Y_true_all - Y_pred_all)**2))
cvrmse = rmse / Y_true_all.mean()
mae = np.mean(np.abs(Y_true_all - Y_pred_all))
r2 = 1 - np.sum((Y_true_all - Y_pred_all)**2) / np.sum((Y_true_all - Y_true_all.mean())**2)
print("RMSE: {:.2f}".format(rmse))
print("CVRMSE: {:.2f}".format(cvrmse))
print("MAE: {:.2f}".format(mae))
print("R2: {:.2f}".format(r2))

RMSE: 7.68
CVRMSE: 0.41
MAE: 5.28
R2: 0.59
